<a href="https://colab.research.google.com/github/TimoFeyKiselevGA/2020-e4-kiselev-Language-Classifier/blob/main/JazykovaClasifikace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1, downloading and extracting texts** 

## Imports and downloading

In [ ]:
import xml.etree.ElementTree as etree
import codecs
import csv
import os
import pandas as pd
import pickle
import re
%load_ext google.colab.data_table
import os
import gc
import psutil
!pip3 install -U scikit-learn

pd.set_option('display.max_colwidth', -1)

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
languages_to_download = {
  "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,

  "UK": True,
  
  "FI": True,
  "BG": True,
  "ES":True,
  "PT":True,
}

languages_links = {
    
    "CZ": "https://dumps.wikimedia.org/cswiki/20210120/cswiki-20210120-pages-articles.xml.bz2",
    "RU": "https://dumps.wikimedia.org/ruwiki/20210101/ruwiki-20210101-pages-articles-multistream5.xml-p3835773p5335772.bz2",
    "FR": "https://dumps.wikimedia.org/frwiki/20210101/frwiki-20210101-pages-articles-multistream2.xml-p306135p1050822.bz2",
    "DE": "https://dumps.wikimedia.org/dewiki/20210101/dewiki-20210101-pages-articles-multistream6.xml-p9261245p10761244.bz2",
    "EN": "https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-articles-multistream9.xml-p2936261p4045402.bz2",
    "PL": "https://dumps.wikimedia.org/plwiki/20210101/plwiki-20210101-pages-articles-multistream4.xml-p1199519p2047892.bz2",
    "IT": "https://dumps.wikimedia.org/itwiki/20210101/itwiki-20210101-pages-articles-multistream4.xml-p2206775p3593336.bz2",
    # "JA": "https://dumps.wikimedia.org/jawiki/20210101/jawiki-20210101-pages-articles-multistream5.xml-p1721647p2807947.bz2",
    "UK": "https://dumps.wikimedia.org/ukwiki/20210120/ukwiki-20210120-pages-articles-multistream4.xml-p987986p1674442.bz2",
    # "AR": "https://dumps.wikimedia.org/arwiki/20210120/arwiki-20210120-pages-articles-multistream4.xml-p2482316p3982315.bz2",
    "FI": "https://dumps.wikimedia.org/fiwiki/20210120/fiwiki-20210120-pages-articles-multistream.xml.bz2",
    "BG": "https://dumps.wikimedia.org/bgwiki/20210120/bgwiki-20210120-pages-articles-multistream.xml.bz2",
    "ES":"https://dumps.wikimedia.org/eswiki/20210320/eswiki-20210320-pages-articles-multistream5.xml-p4566829p6066828.bz2",
    "PT":"https://dumps.wikimedia.org/ptwiki/20210320/ptwiki-20210320-pages-articles-multistream6.xml-p5024909p6524729.bz2",
}

languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', "UK", "FI", "BG","ES","PT"]
languages_to_process = {
  "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,

  "UK": True,
  
  "FI": True,
  "BG": True,
  "ES":True,
  "PT":True,
}

In [ ]:
def rename(s):
    return languages_links[s].split("/")[len(languages_links[s].split("/"))-1].split(".")[0]+ "." +languages_links[s].split("/")[len(languages_links[s].split("/"))-1].split(".")[1]

In [ ]:
for lang in languages:
    if languages_to_download[lang]:
        print("downloading " + lang)
        os.system("wget " + languages_links[lang])
        print("extracting " + lang)
        os.system("bzip2 -d ./ " + languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1])
        os.system("mv " + rename(lang) + " " + languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1].split(".")[0] + ".xml")


downloading CZ
extracting CZ
downloading RU
extracting RU
downloading FR
extracting FR
downloading DE
extracting DE
downloading EN
extracting EN
downloading PL
extracting PL
downloading IT
extracting IT
downloading UK
extracting UK
downloading FI
extracting FI
downloading BG
extracting BG
downloading ES
extracting ES
downloading PT
extracting PT


## Creating pkl files with texts without most special chars. from xmls.

In [ ]:
def strip_tag_name(t):
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [ ]:
def articles_to_csv(totalCount):
    with codecs.open(pathTexts, "w", ENCODING) as textsFH:
        textWriter = csv.writer(textsFH, quoting=csv.QUOTE_MINIMAL)
        textWriter.writerow(['text'])
        for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
            tname = strip_tag_name(elem.tag)
            if event == 'start':
                if tname == 'page':
                    title = ''
                    id = -1
                    redirect = ''
                    inrevision = False
                    ns = 0
                    text = ''
                elif tname == 'text':
                    text = elem.text
                    textWriter.writerow([text])
                elif tname == 'page':
                    totalCount += 1
                if totalCount > 1 and (totalCount % 100000) == 0:
                    print("{:,}".format(totalCount))
            elem.clear()

In [ ]:
for lang in languages:
    if languages_to_process[lang]:
        print(lang)
        PATH_WIKI_XML = './'
        FILENAME_WIKI = languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1].split('.')[0] + '.xml'
        texts_template = 'temp.csv'
        ENCODING = "utf-8"
        pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
        pathTexts = os.path.join(PATH_WIKI_XML, texts_template)
        totalCount = 0
        articleCount = 0
        title = None
        articles_to_csv(totalCount)
        a = pd.read_csv('temp.csv')
        a = a.dropna()
        a['text'] = a['text'].str.replace(',|\$|\{|\}|\-|\+|\[|\]|\\n|\<|\>|\"|\=|\:|\;|\*|\||\#|\$|\(|\)|\/|\.|\–|\_|\«|\»|\—|\!|\&|\?|\`|\~|\%|\@|\¡|\™|\£|\¢|\∞|\§|\¶|\•|\ª|\º|\–|\≠|\«|\æ|\÷|\≥|\≤|\÷|\»|\\|\^|\ˆ|\|\\|\||\）|\（|\、|\'', ' ')
        a['text'] = a['text'].str.replace('\d+', '')
        a['text'] = a['text'].str.replace('\s\s+' , ' ')
        a = a.drop([a.index[0]])
        a = a.reset_index(drop=True)
        a['text'] = a['text'].str.lower()
        a.to_pickle("./texts_" + lang +  ".pkl")
        

CZ
RU
FR
DE
EN
PL
IT
UK
FI
BG
ES
PT


In [ ]:
%reset -f

# **Part 2: Dataset preparation**

## Imports and settings

In [ ]:
!pip3 install -U scikit-learn
import xml.etree.ElementTree as etree
import codecs
import csv
import os
import pandas as pd
import pickle
import re
%load_ext google.colab.data_table
import os
import gc
import psutil
import numpy as np
import random
#random seeds
random.seed(0)
np.random.seed(0)

pd.set_option('display.max_colwidth', -1)

languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', "UK", "FI", "BG","ES","PT"]
languages_to_process = {
   "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,

  "UK": True,
  
  "FI": True,
  "BG": True,
  "ES":True,
  "PT":True,
}

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.1)
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Dataset creation

In [ ]:
def chunkstring(string, length):
    return re.findall('.{%d}' % length, string)

def toPickleLoadList(string, textList):
    file_name = "texts_" + string + "_prep.pkl"
    open_file = open(file_name, "wb")
    pickle.dump(textList, open_file)
    open_file.close()
    print("done " + lang)

def fromPickleLoadList(string):
    file_name = "texts_" + string + "_prep.pkl"
    open_file = open(file_name, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

def chunkDataFrames(lang):
    print("start " + lang)
    df = pd.read_pickle("./texts_"+ lang + ".pkl")
    df = df[df['text'].apply(lambda x: len(x)>100)]
    # df = df[~df.text.str.startswith(' redirect', na=True)]
    df.text = df.text.replace('\s+', ' ', regex=True)
    df = df.text.str.cat(sep='')
    textList = chunkstring(df, 200)
    toPickleLoadList(lang, textList)
    del df;  gc.collect()
    del textList;  gc.collect()


In [22]:
for lang in languages:
    if languages_to_process[lang]:
        chunkDataFrames(lang)
    

start CZ
done CZ
start RU
done RU
start FR
done FR
start DE
done DE
start EN
done EN
start PL
done PL
start IT
done IT
start UK
done UK
start FI
done FI
start BG
done BG
start ES
done ES
start PT
done PT


In [23]:
from sklearn.utils import shuffle
text_to_tokens = []
id_lang = []
id = 0
delka = 100000

for lang in languages:
    df = shuffle(fromPickleLoadList(lang))
    df = df[:delka]
    id_lang = id_lang + [id] * delka
    id = id + 1
    text_to_tokens = text_to_tokens + df

In [24]:
data_tuples = list(zip(text_to_tokens,id_lang))
df = pd.DataFrame(data_tuples, columns=['X','Y'])

In [25]:
df = shuffle(df)

In [26]:
df.head()

,X,Y
705082,ана особистість портал мистецтво вибрана особистість br портал мистецтво вибрана особистість портал мистецтво вибрана особистість br портал мистецтво вибрана особистість портал мистецтво вибрана особи,7
657029,bawinkel coa png voce stemma stato deu grado amministrativo tipo comuni della germania comune apparten a un comunità amministrative della germania samtgemeinde divisione amm grado bassa sassonia divis,6
962911,по късно отказаната балиситична ракета ур става основа за разработването на ракети протон ракета „протон“ ур основна ур ур е междуконтинентална балистична ракета мкбр тя е течногоривна има степени из,9
930088,solea plebeia е вид лъчеперка от семейство pleuronectidae разпространение видът е разпространен в нова зеландия северен остров нова зеландия северен остров и южен остров нова зеландия южен остров ref,9
830942,ieto isbn www https finna fi record alli kirjaviite tekijä kivelä simo k nimeke matlab opas vuosi selite sivua julkaisupaikka helsinki julkaisija otatieto isbn isbn www https finna fi record alli kirj,8


# Part 3: Model training

## Imports and settings


In [27]:
!pip3 install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.1)


In [28]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score


## Model creation and training

In [30]:
def simple_tokenizer(txt, min_len=3):
    all_tokens = re.compile(r'[\w\d]+').findall(txt)
    all_tokens = [x.lower() for x in all_tokens]
    return [token for token in all_tokens if len(token) >= min_len]

In [49]:
sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=simple_tokenizer,
                                                      max_df=0.8,
                                                      min_df=50)),
                             ('cls', LogisticRegression())))

In [50]:
sklearn_pipeline.fit(train['X'].to_numpy(), train['Y'].to_numpy())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.8, min_df=50,
                                 tokenizer=<function simple_tokenizer at 0x7f11f71ef0e0>)),
                ('cls', LogisticRegression())])

In [51]:
sklearn_train_pred = sklearn_pipeline.predict_proba(train['X'].to_numpy())
sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(train['Y'].to_numpy()))

In [52]:
print('train loss', float(sklearn_train_loss))
print('acc', accuracy_score(train['Y'], sklearn_train_pred.argmax(-1)))
print()

train loss 1.6938838281427884
acc 0.9702821428571429



In [53]:
sklearn_test_pred = sklearn_pipeline.predict_proba(test['X'].to_numpy())
sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(test['Y'].to_numpy()))

In [54]:
print('Val loss', float(sklearn_test_loss))
print('acc', accuracy_score(test['Y'], sklearn_test_pred.argmax(-1)))

Val loss 1.6985612352349602
acc 0.9648472222222222


In [55]:
pokus = ['hilft zu lernen oder.', 'no jo ale to neni  ', ' s a song by the', 'Существуют оба потом']
for i in range(0, len(pokus)):
    print(languages[sklearn_pipeline.predict_proba(pokus).argmax(-1)[i]])

DE
CZ
EN
RU


In [56]:
languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', "UK", "FI", "BG","ES","PT"]

In [57]:
pickle.dump(sklearn_pipeline, open("model.pkl", 'wb'))

In [58]:
loaded_model = pickle.load(open("model.pkl", 'rb'))

In [61]:
pokus = ['hilft zu lernen oder.', 'jak se mas ne', ' s a song by the',
         'Существуют оба потом', 'Южнославянски език, написан на кирилица',
         'slaavilainen kieli, joka']
for i in range(0, len(pokus)):
    print(loaded_model.predict_proba(pokus)[i])
    print(languages[loaded_model.predict_proba(pokus).argmax(-1)[i]])

[0.02311442 0.02551317 0.01361722 0.77252467 0.01041237 0.03600793
 0.01993936 0.01690678 0.02097822 0.02701301 0.0187936  0.01517924]
DE
[0.18855176 0.01562587 0.0130478  0.0104378  0.00701823 0.61251271
 0.01403201 0.01172365 0.0101429  0.01116703 0.00785551 0.09788475]
PL
[0.00519594 0.01862926 0.00212957 0.00338307 0.92470914 0.00442879
 0.00695726 0.00448965 0.00343128 0.00441805 0.01927655 0.00295145]
EN
[0.07359245 0.35296746 0.05954584 0.04620306 0.03242376 0.0803891
 0.05131163 0.06527225 0.07535875 0.06482333 0.04877548 0.04933688]
RU
[0.0054035  0.00443304 0.00394328 0.00342354 0.00186392 0.00588054
 0.00386802 0.00410648 0.005397   0.95465336 0.0036231  0.00340422]
BG
[3.20796101e-06 3.41268611e-06 3.15071612e-06 2.17655151e-06
 1.24040664e-06 3.73824407e-06 2.58567133e-06 2.95229766e-06
 9.99969399e-01 3.28955517e-06 2.44617867e-06 2.40034697e-06]
FI


## generate django secret key

In [60]:
from django.core.management.utils import get_random_secret_key

print(get_random_secret_key())

tc*$9v+fc2w85-#px@_d=t7$hii8g-xtpsvo5bb$*^#=#9910m
